In [1]:
# Import packages

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, accuracy_score

import matplotlib.pyplot as plt

import pickle

import seaborn as sns

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

/Users/boramert/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/boramert/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
def drop_infs(ds):
    ds.replace([np.inf, -np.inf], np.nan, inplace=True)
    ds.dropna(how='any', inplace=True)
    return ds

In [3]:
print("Loading Dataset...")
dataset = pd.read_csv("test_dataset.csv")
dataset = drop_infs(dataset)
print("Dataset Loaded.")
print("_____________________________________________________________________")

Loading Dataset...
Dataset Loaded.
_____________________________________________________________________


In [4]:
labels = dataset[' Label'].copy()
print("Labels:")
print(np.unique(labels))
print("_____________________________________________________________________")

Labels:
['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator' 'Web Attack � Brute Force' 'Web Attack � Sql Injection'
 'Web Attack � XSS']
_____________________________________________________________________


In [5]:
xgb_labels = dataset[' Label'].copy()
le = preprocessing.LabelEncoder()
xgb_labels = le.fit_transform(xgb_labels)
print("Labels for XGBoost Classification:")
print(np.unique(xgb_labels))
print("_____________________________________________________________________")

Labels for XGBoost Classification:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
_____________________________________________________________________


In [6]:
dataset.loc[dataset[' Label'] == 'BENIGN', ' Label'] = 0  # Saldırılar için 1 normaller için 0 değeri verme
dataset.loc[dataset[' Label'] != 0, ' Label'] = 1
autoencoder_labels = dataset[' Label'].copy()
print("Labels for Autoencoder Classification:")
print(np.unique(autoencoder_labels))
print("_____________________________________________________________________")

Labels for Autoencoder Classification:
[0 1]
_____________________________________________________________________


In [7]:
# Dataseti X ve Y ayırma
x = dataset.drop(' Label', axis=1)
y = np.c_[autoencoder_labels, xgb_labels]
# Normal ve anomali ayırma
x_normal = x[y == 0]
x_anomaly = x[y == 1]

In [8]:
# Autoencoder Train-Test datası ayırma
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=12345,
                                                    stratify=y)

In [9]:
# Scaling
scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
y_xgb = y_test[:, 1]
y_test = y_test[:, 0]
y_train = y_train[:, 0]
x_train = x_train[y_train == 0]
y_train = y_train[y_train == 0]

In [10]:
# Autoencoder
input = tf.keras.layers.Input(shape=(78,))
encoder = tf.keras.Sequential([
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu')])(input)
decoder = tf.keras.Sequential([
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(78, activation='sigmoid')])(encoder)
autoencoder = tf.keras.Model(inputs=input, outputs=decoder)
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

2022-12-18 16:21:33.448586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Autoencoder eğitme
print("Autoencoder Training...")
history = autoencoder.fit(x_train, x_train,
                          epochs=11,
                          batch_size=20,
                          validation_split=0.2,
                          shuffle=True)
print("Training Complete.")
print("_____________________________________________________________________")

Autoencoder Training...
Epoch 1/11
66996/72683 [==========================>...] - ETA: 3s - loss: 0.6171

In [ ]:
# Test datasını reconstruct etme
reconstruct = autoencoder.predict(x_test)
print("_____________________________________________________________________")
reconstruct_loss = tf.keras.losses.mse(reconstruct, x_test)
threshold = np.percentile(reconstruct_loss, 85)
print("Threshold: ", threshold)
print("_____________________________________________________________________")

In [ ]:
def predict(model, data, threshold):
    # reconstructions = model(data)
    reconstructions = model.predict(data)
    loss = tf.keras.losses.mse(reconstructions, data)
    prediction = []
    for i in range(len(loss)):
        if loss[i] >= threshold:
            prediction.append(1)
        else:
            prediction.append(0)
    prediction = np.array(prediction)
    return prediction


def print_stats(predictions, labels):
    print("Accuracy = {}".format(accuracy_score(labels, predictions)))
    print("Precision = {}".format(precision_score(labels, predictions)))
    print("Recall = {}".format(recall_score(labels, predictions)))

In [ ]:
preds = predict(autoencoder, x_test, threshold)
print("Preds:")
print(preds[0:10])
print("y_test:")
print(y_test[0:10])
print("Model Stats: ")
print("_____________________________________________________________________")
print_stats(preds.astype(bool), y_test.astype(bool))
print("_____________________________________________________________________")

In [ ]:
# XGBoost data hazırlama
xgb_x = []
xgb_y = []
for i in range(len(x_test)):
    if preds[i] == 1 and y_xgb[i] != 0:
        xgb_x.append(x_test[i, :])
        xgb_y.append(y_xgb[i])
xgb_x = np.array(xgb_x)
xgb_y = np.array(xgb_y)

a = le.inverse_transform(xgb_y)
le2 = preprocessing.LabelEncoder()
xgb_y = le2.fit_transform(a)

In [ ]:
np.unique(xgb_y)

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(xgb_x, xgb_y,
                                                        test_size=0.25,
                                                        random_state=12345)

In [ ]:
attack_predictor = xgb.XGBClassifier(n_estimators=100,
                                     max_depth=1, learning_rate=0.2, verbosity=0,
                                     objective='multi:softmax',
                                     use_label_encoder=False)

In [ ]:
print("XGBoost Cross Validation...")
cv = RepeatedStratifiedKFold(n_splits=5, random_state=12345)
n_scores = cross_val_score(attack_predictor, x_train2, y_train2, scoring='accuracy',
                           cv=cv, error_score='raise')
print("CV Complete.")

In [ ]:
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
print("____________________________________________ \n \n")

In [ ]:
print("XGBoost Training...")
attack_predictor.fit(x_train2, y_train2)
print("Training Complete.")
attack_preds = attack_predictor.predict(x_test2)

attack_predictor.score(x_test2, y_test2)

In [ ]:
print("Attack Type Classification")
print("")
print("Classification Report: ")
print(classification_report(y_test2, attack_preds))

In [ ]:
print("")
print("Accuracy Score: ", accuracy_score(y_test2, attack_preds))
print("____________________________________________ \n \n")

In [ ]:
a = np.unique(y_test2)
a = le2.inverse_transform(a)

In [ ]:
LABELS = a
conf_matrix = confusion_matrix(y_test2, attack_preds)
plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS,
            yticklabels=LABELS, annot=True, fmt="d")
plt.title("Attack Type Classification")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
print("Pickling Models...")
pickle.dump(scaler, open('scaler.pkl', 'wb'))
pickle.dump(autoencoder, open('autoencoder.pkl', 'wb'))
pickle.dump(attack_predictor, open('attack_predictor.pkl', 'wb'))
print("Pickle Complete.")